In [1]:
# Meshing
from meshing import process
            
process(data_path="data/models", output_path="data/meshes", sort=False, rescale=False, 
            filter_level=0, precision=[0.95, 0.04], subfolders=False)

ModuleNotFoundError: No module named 'OCC'

In [33]:
# Preprocessing/Create gray patterns

import patterns as pa
import matplotlib


# Initialize textures
def init_textures(tex_size=[1920, 1080]):
    gray = pa.genboard_graycode(proj_resolution=tex_size)
    textures_h = np.swapaxes(gray[0][:, :], 0, 2)
    textures_v = np.swapaxes(gray[1][:, :], 0, 2)
    white = np.ones(tex_size[::-1], dtype=np.uint8)*255
    black = np.ones(tex_size[::-1], dtype=np.uint8)

    patterns = [white, black]
    for t in textures_v:
        patterns.append(t)
        patterns.append(255-t)
    for t in textures_h:
        patterns.append(t)
        patterns.append(255-t)
        
    for p_i, p in enumerate(patterns):
        pat = Image.fromarray(p)
        pat.save("rendering/patterns/gray/default/img_%02i.png"%p_i)

textures = init_textures()



def load_images(prefix, pars, mm=[1, 39], typ="256"):
    images = []
    for i in range(mm[0], mm[1]):
        filename_r = "%s/%s/%s/%s_%02i.png"%(pars["root_folder"], pars["img_folder"], typ, prefix, i)
        im = matplotlib.image.imread(filename_r)
        img = rgb2gray(im)*255

        #print(img.shape, img.dtype, np.min(img), np.max(img))
        images.append(img)
    return images